In [ ]:
import sys

PATH = "../../.."
sys.path.append(f'{PATH}/')
from src.utils.pyspark_init import set_pySpark
import pyspark.pandas as ps
from src.dataProcessing.tweet.Tweet import Tweet
from src.utils.globalVariable import REGULAR_ACTIVE_USER_DIR, TWEET_DATABASE_DIR
from src.utils.slidingWindow import get_period_list, utc_to_string

set_pySpark(memory='30g')

In [ ]:
def save_Tweet(PERIOD_DIR, startPeriod_utc, endPeriod_utc):
    user_id = ps.read_parquet(f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/user")["id"]
    tweet = Tweet(f"{PATH}/{TWEET_DATABASE_DIR}")
    
    socialTweet = tweet.get_socialTweet_period(startPeriod_utc, endPeriod_utc)
    socialTweet = socialTweet[socialTweet["user_id"].isin(user_id.values)]
    socialTweet.to_parquet(f"{PERIOD_DIR}/tweet/socialTweet")
    
    allTweet = tweet.get_tweet_in_period(startPeriod_utc, endPeriod_utc)
    allTweet = allTweet[allTweet["user_id"].isin(user_id.values)]
    allTweet.to_parquet(f"{PERIOD_DIR}/tweet/allTweet")

In [3]:
period_list = get_period_list()
for first_period, second_period in period_list:
    first_period_str = utc_to_string(first_period)
    second_period_str = utc_to_string(second_period)

    PERIOD_DIR = f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/{first_period_str}-{second_period_str}"
    save_Tweet(PERIOD_DIR, first_period, second_period)